### Least Squares Monte Carlo for pricing American-style options

Here, we implement the Least Square Monte Carlo simulation method described by Longstaff-Schwartz (2001). We will assume geometric brownian motion and volatility forecasts from the GARCH model. Afterwards, we will use the simulation to price derivatives of SPX and compare to market prices at the time.

In [303]:
import numpy as np
import pandas as pd
from scipy.special import comb
from sklearn.linear_model import LinearRegression

In [304]:
s = 1
n = 8
r = 0.06
sigma = 0.2
T = 5
r_daily = r / 365
sigma_daily = sigma / 365**0.5
type = 'put'

In [305]:
def initiate_path(npaths = 8, s = 100, r = 0.06, sigma = 0.2, T = 5, strike = 100, type='put'):

    path = pd.DataFrame(np.zeros(npaths) + s)
    r_daily = r/365
    sigma_daily = sigma/365**0.5

    for i in range(T):
        day = path.shape[1] - 1
        current = path.iloc[:,day]
        next = current * np.exp(r_daily - sigma_daily**2/2 + sigma_daily * np.random.normal(size = npaths))
        path = pd.concat([path, next], axis=1)

    path.columns = list(range(T+1))

    if type == 'put':
        path['gain_at_maturity'] = strike - path[T]
    else:
        path['gain_at_maturity'] = strike - path[T]
    
    path['gain_at_maturity'] = path['gain_at_maturity'].apply(lambda x:max(x, 0))
    path['value_at_maturity'] = path['gain_at_maturity']
    path['executed'] = T
    return path

In [306]:
path = initiate_path()
path

,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed
0,100.0,100.793748,99.249520,99.151946,98.065424,98.749133,1.250867,1.250867,5
1,100.0,100.113533,100.230247,99.649351,100.397113,101.916261,0.000000,0.000000,5
2,100.0,100.308777,99.366166,99.196070,99.603794,100.409063,0.000000,0.000000,5
3,100.0,100.376370,100.418442,102.144058,103.762460,103.073803,0.000000,0.000000,5
4,100.0,99.237478,100.082924,101.069904,100.264646,100.941486,0.000000,0.000000,5
5,100.0,100.634133,101.288051,101.521106,102.750313,103.614319,0.000000,0.000000,5
6,100.0,99.853268,99.216642,97.911987,96.555754,96.480068,3.519932,3.519932,5
7,100.0,100.057415,99.361532,99.475706,99.139700,98.793142,1.206858,1.206858,5


In this table, each row represents one of the simulated paths of the underlying asset's price over the specified period (5 days). Columns 0 to 5 represent the simulated asset prices over 5 days, for each row, it starts with the initial asset price and changes over time based on the specified drift and volatility, assuming geometric brownian motion. 
The gain_at_maturity column shows the payoff of the option as the difference between the asset and strike prices, but only if it is below the strike price.
Here, they are all executed on the last day because we have not yet implement the early execution decision logic. 

In [307]:
reg = LinearRegression()

# for generation of polynomial series

def polynomial(n, data):
    result = data
    if n == 1:
        return result.values.reshape(-1, 1)
    for i in range(2, n+1):
        result = pd.concat([result, data**i], axis = 1)
    return result


In [308]:
def continuation_values(i, path = path, basis = polynomial, n=1, display=True, strike=100, option_type='put', r=0.06):
    r_daily = r / 365
    if 'executed_on' not in path.columns:
        path['executed_on'] = 'not executed'
    
    day = path[[i, 'value_at_maturity']].copy()
    if option_type == 'put':
        day = day[day[i] < strike]
    else:
        day = day[day[i] > strike]
    
    if day.shape[0] == 0:
        return
    
    transform = basis(n=n, data=day[i])
    reg = LinearRegression()
    reg.fit(transform, day['value_at_maturity'])
    
    if display:
        print('\nDay', i, 'ITM path:', list(day.index))
        print('Conditional expectation function:', 'intercept:', reg.intercept_, 'coef:', reg.coef_)
    
    day['expected_continuation_value'] = reg.predict(transform) * (1 / (1 + r_daily))
    
    if option_type == 'put':
        day['execute_value'] = day[i].apply(lambda x: max(0, strike - x))
    else:
        day['execute_value'] = day[i].apply(lambda x: max(0, x - strike))
    
    day['executed'] = day['execute_value'] > day['expected_continuation_value']
    
    if display:
        print('\nChoose to execute on these paths:', day[day['executed']==True].index.tolist())
    
    def update(row):
        if row['executed']:
            return row['execute_value'] * (1 + r_daily) ** (T - i)
        else:
            return row['value_at_maturity']
    
    day['new_value_at_maturity'] = day.apply(update, axis=1)
    path.loc[day.index, 'value_at_maturity'] = day['new_value_at_maturity']
    path.loc[day[day['executed'] == True].index, 'executed_on'] = i


In [309]:
for day in range(T-1, 0, -1):
    continuation_values(day)
    display(path)


Day 4 ITM path: [0, 2, 6, 7]
Conditional expectation function: intercept: 102.8798559726033 coef: [-1.03095625]

Choose to execute on these paths: [0, 2, 6, 7]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.793748,99.249520,99.151946,98.065424,98.749133,1.250867,1.934894,5,4
1,100.0,100.113533,100.230247,99.649351,100.397113,101.916261,0.000000,0.000000,5,not executed
2,100.0,100.308777,99.366166,99.196070,99.603794,100.409063,0.000000,0.396271,5,4
3,100.0,100.376370,100.418442,102.144058,103.762460,103.073803,0.000000,0.000000,5,not executed
4,100.0,99.237478,100.082924,101.069904,100.264646,100.941486,0.000000,0.000000,5,not executed
5,100.0,100.634133,101.288051,101.521106,102.750313,103.614319,0.000000,0.000000,5,not executed
6,100.0,99.853268,99.216642,97.911987,96.555754,96.480068,3.519932,3.444812,5,4
7,100.0,100.057415,99.361532,99.475706,99.139700,98.793142,1.206858,0.860442,5,4



Day 3 ITM path: [0, 1, 2, 6, 7]
Conditional expectation function: intercept: 186.2363432674472 coef: [-1.86631647]

Choose to execute on these paths: [1]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.793748,99.249520,99.151946,98.065424,98.749133,1.250867,1.934894,5,4
1,100.0,100.113533,100.230247,99.649351,100.397113,101.916261,0.000000,0.350764,5,3
2,100.0,100.308777,99.366166,99.196070,99.603794,100.409063,0.000000,0.396271,5,4
3,100.0,100.376370,100.418442,102.144058,103.762460,103.073803,0.000000,0.000000,5,not executed
4,100.0,99.237478,100.082924,101.069904,100.264646,100.941486,0.000000,0.000000,5,not executed
5,100.0,100.634133,101.288051,101.521106,102.750313,103.614319,0.000000,0.000000,5,not executed
6,100.0,99.853268,99.216642,97.911987,96.555754,96.480068,3.519932,3.444812,5,4
7,100.0,100.057415,99.361532,99.475706,99.139700,98.793142,1.206858,0.860442,5,4



Day 2 ITM path: [0, 2, 6, 7]
Conditional expectation function: intercept: 1663.8539271523414 coef: [-16.73938082]

Choose to execute on these paths: [2, 7]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.793748,99.249520,99.151946,98.065424,98.749133,1.250867,1.934894,5,4
1,100.0,100.113533,100.230247,99.649351,100.397113,101.916261,0.000000,0.350764,5,3
2,100.0,100.308777,99.366166,99.196070,99.603794,100.409063,0.000000,0.634147,5,2
3,100.0,100.376370,100.418442,102.144058,103.762460,103.073803,0.000000,0.000000,5,not executed
4,100.0,99.237478,100.082924,101.069904,100.264646,100.941486,0.000000,0.000000,5,not executed
5,100.0,100.634133,101.288051,101.521106,102.750313,103.614319,0.000000,0.000000,5,not executed
6,100.0,99.853268,99.216642,97.911987,96.555754,96.480068,3.519932,3.444812,5,4
7,100.0,100.057415,99.361532,99.475706,99.139700,98.793142,1.206858,0.638783,5,2



Day 1 ITM path: [4, 6]
Conditional expectation function: intercept: -555.1485808193396 coef: [5.59414235]

Choose to execute on these paths: [4]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.793748,99.249520,99.151946,98.065424,98.749133,1.250867,1.934894,5,4
1,100.0,100.113533,100.230247,99.649351,100.397113,101.916261,0.000000,0.350764,5,3
2,100.0,100.308777,99.366166,99.196070,99.603794,100.409063,0.000000,0.634147,5,2
3,100.0,100.376370,100.418442,102.144058,103.762460,103.073803,0.000000,0.000000,5,not executed
4,100.0,99.237478,100.082924,101.069904,100.264646,100.941486,0.000000,0.763023,5,1
5,100.0,100.634133,101.288051,101.521106,102.750313,103.614319,0.000000,0.000000,5,not executed
6,100.0,99.853268,99.216642,97.911987,96.555754,96.480068,3.519932,3.444812,5,4
7,100.0,100.057415,99.361532,99.475706,99.139700,98.793142,1.206858,0.638783,5,2


In [310]:
# option value on day 0
print('Simulated price under default setting:')
path['value_at_maturity'].mean() * (1+r_daily)**(-T)

Simulated price under default setting:


0.970005303115134

In [366]:
def option_price(basis = polynomial, n = 1, display = False, npaths = 10000, s = 100, r = 0.06, sigma =0.2, T = 5, strike = 100, type = 'put'):
    path = initiate_path(npaths=npaths, s=s, r=r, sigma=sigma, T=T, strike=strike, type=type)

    r_daily = r / 365
    T = path.shape[1] - 3

    for day in range(T-1, 0, -1):
        continuation_values(i=day, path=path, basis=basis, n=n, strike=strike, option_type=type, display=False)

    if display:
        display(path)

    discounted_values = path['value_at_maturity'] * (1 + r_daily) ** (-T)
    mean_val = np.mean(discounted_values)

    return mean_val

## Pricing Compared to Market Data

The S&P 500 closed at 3,852.97 on January 4, 2023. Looking at the SPX option prices at the time, we see that the put contract expiring on February 2, 2023, for a strike price of 3750, had a bid of 49.1 and an ask of 49.8 (data from optionsDX). 
The US 1 Month Treasury bond yield was 4.20%. 
Our forecasted volatility was around 0.22 at that time.


In [382]:
mean_value, stddev_value = option_price(basis = polynomial, n = 2, strike = 3750, type='put', s = 3852.97, T = 30, r = 0.042, sigma = 0.22, npaths = 10000)
print(f"Mean Option Price using 2 term polynomial basis with 10,000 paths: ${mean_value:.2f}")

Mean Option Price using 2 term polynomial basis with 10,000 paths: $49.49
